<a href="https://colab.research.google.com/github/harrisb002/CS_480/blob/main/Projects/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cylindrical Board Queen Placement
This code aims to solve the problem of placing as many additional queens on a cylindrical chessboard such that no queens attack each other, using a recursive version of Depth-First Search (DFS).


### Helper Functions

In [ ]:
def no_more_queens_can_be_added(board, n):
    """
    Check if no more queens can be added to board.

    Parameters:
    board (list): The current board.
    n (int): The dimension of the board (n x n).

    Returns:
    bool: True if no more queens can be added, otherwise False.
    """
    # Check if all rows have a queen placed
    for row in range(n):
        if board[row] == -1:  # Row is still empty.
            return False
    return True  # Rows are filled; no more queens can be placed.

In [ ]:
def is_safe(board, row, col, n):
    """
    Check if safe to place a queen on a square.

    Parameters:
    board (list): The current board.
    row (int): The row index where the queen is going to be placed.
    col (int): The column index where the queen is going to be placed.
    n (int): Board dimension.

    Returns:
    bool: True if safe to place queen on square, false otherwise
    """
    # Check all prev. queens for attacks.
    for r in range(len(board)):
        if board[r] == -1:
            continue  # Skip rows with no queens.

        # Vertical attack: Queen is in the same column.
        if board[r] == col:
            return False

        # Diagonal attack: Queen is on the same diagonal.
        if abs(board[r] - col) == abs(r - row):
            return False

        # Cylindrical diagonal attack:
        # Checking if the queen at `(row, col)`
        # would be attacked diagonally by a prev. placed queen at (r, board[r])

        # Handling cases where columns wrap around from 0 to n-1
        #   making sure that neg. differences are absolute valued.
        if (board[r] - col + n) % n == abs(r - row):
            return False

        # Checking the opposite diagonal direction
        if (col - board[r] + n) % n == abs(r - row):
            return False

    return True

### Queen Depth First Search
- Focuses on solving a variant of the n-queens problem where the chessboard is cylindrical, such that the first and last columns are adjacent.
- My goal is to place as many additional queens as possible on a partially filled board without any queens attacking each other.
- This involves a Depth-First Search (DFS) solution in which the program handles input that may skip rows and reports the number of nodes explored during the DFS process.


- B[i]: Column of a queen in row
- -1:   No queen is present in that row.
- Retains existing queens from the input and places additional queens in remaining rows, replacing the -1 values.
- Skips rows and handles non-sequential queen placements.
- Track and report the total # of nodes expanded

In [ ]:
def c_queensDFS(board, n, nodes_expanded, placed_queens, max_queens):
    """
    Recursive DFS function to solve the cylindrical n-queens problem.

    Parameters:
    board (list): The current board where each index represents a row,
                  the value at each index represents the column where a queen is placed.
                  A value of -1 means that no queen is placed in that row.
    n (int): The dimension of the chessboard (n x n).
    nodes_expanded (list): A single-element list used to track the total number of nodes expanded during DFS.
    placed_queens (int): The current number of queens placed on the board.
    max_queens (int): The max number of queens that can be placed on the board.

    Returns:
    tuple: A tuple with the max number of queens placed and the board.
    """

    # Base Case: Stop search if no more queens can be placed on board.
    if no_more_queens_can_be_added(board, n):
        return placed_queens, board  # Return current count of placed queens and final board.

    count = placed_queens  # INit the count of placed queens.
    best_solution = board[:]  # Shallow copy of board

    # Go through each row of board.
    for row in range(n):
        if board[row] != -1:  # Skip rows that already have a queen.
            continue

        # Try and place a queen in each column of current row.
        for col in range(n):
            if is_safe(board, row, col, n):  # Check if current square is safe.
                board[row] = col  # Place queen in current position.

                nodes_expanded[0] += 1  # Increment counter for expanded nodes.

                # Recursively try to place another queen
                temp_count, temp_board = c_queensDFS(board, n, nodes_expanded, placed_queens + 1, max_queens)

                # If placing the queen results in a better solution (more queens placed), update the best solution.
                if temp_count > count:
                    count = temp_count
                    best_solution = temp_board[:]

                # Backtrack: Remove the queen and try the next possibility.
                board[row] = -1

        # Early stopping: Stop the search if the maxnum of queens have been placed.
        if count == max_queens:
            break

    return count, best_solution

### Main

In [ ]:
def main():
    """
    Execute cylindrical n-queens DFS solution and print results.

    Input board is defined within this function, and DFS is initiated to find max
    num of queens that can be placed on the cylindrical chessboard.
    """
    # Init chessboard from the user
    board = input("Enter the board config (comma-separated values, -1 for empty rows): ")

    # Remove square brackets and whitespace, then split by commas and convert to ints
    board = list(map(int, board.strip().replace('[', '').replace(']', '').split(',')))

    n = len(board)  # Get size of board. (Should be 8x8 ideally)

    placed_queens = sum(1 for q in board if q != -1) # Initial num of queens

    max_queens = n  # Max num of queens is the size of the board.

    nodes_expanded = [1]  # Init counter for num of nodes expanded (start with root)

    # Start DFS to find the max num of queens that can be placed.
    max_queens_placed, best_solution = c_queensDFS(board, n, nodes_expanded, placed_queens, max_queens)

    print(f"Output: {best_solution, nodes_expanded[0]}")

if __name__ == "__main__":
    main()

Enter the board config (comma-separated values, -1 for empty rows): [-1, -1, -1, 3, -1, -1, -1, -1]
Output: ([1, 4, 0, 3, -1, 2, 5, -1], 66373)


#### Current Test Cases (Need More)
- Input = [-1, -1, -1, 3, -1, -1, -1, -1]
  - Output: (66373, [1, 4, 0, 3, -1, 2, 5, -1])
- Input: [-1, -1, 1, 5, -1, -1, -1, -1]
  - Output: (1301, [4, 6, 1, 5, 2, 0, -1, -1])